# Dynamic bifurcation of consensus

We consider the fast-slow system:

\begin{equation}\label{appendix_fast_slow}
\begin{aligned}
\dot{\mathbf{x}} &= -D\mathbf{x} + A\mathbf{S}(\mathbf{x}, u) \\
\dot{u} &= -\varepsilon .
\end{aligned}
\end{equation}

In [ ]:
import networkx as nx
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
import pylab
import matplotlib2tikz as mplt2tikz
import net_sym as ns

In [ ]:
g = ns.get_strongly_connected_digraph(10)
# To take a graph from X, comment the line above
# and uncomment the lines below
# g = nx.DiGraph()
# g.add_edges_from(([1,0],[0,2],[2,1]))
N = nx.number_of_nodes(g)
A = nx.adjacency_matrix(g)
D = np.diag(np.asarray(np.sum(A, axis=1)).reshape(-1))

In the system given by \cref{appendix_fast_slow}, we take $[\mathbf{S}(\mathbf{x},u)]_i = S(x_i,u)$ and 

\begin{equation}
S(x,u) := x + u + x^2.
\end{equation}

In [ ]:
s_xu = lambda x,u: x + u + np.square(x)
phi_xu = lambda x,u: s_xu(x,u) - x

x_range = np.arange(-2.1, 2.1, 0.01)

In [ ]:
u = 0
plt.figure(figsize=(14,5))

plt.subplot(121)
plt.plot(x_range, s_xu(x_range, u), linestyle='-', color="black")
plt.plot(x_range, x_range, linestyle='--', color='red')
plt.xlabel(r'$x$')
plt.ylabel(r'$S(x,0.5)$')
plt.grid()

plt.subplot(122)
plt.plot(x_range, -phi_xu(x_range, u), linestyle='-', color="black")
plt.plot(x_range, np.repeat(0,np.size(x_range)), linestyle='--', color='red')
plt.xlabel(r'$x$')
plt.ylabel(r'$\phi(x,0.5)$')
plt.grid()

To integrate the system \cref{appendix_fast_slow}, we use $\varepsilon = 0.1$.

In [ ]:
dt = 0.01
epsilon = 0.1
u_range = np.arange(0.7, 1.5, dt)
n_iter = 1000
x_n = np.zeros((np.size(u_range), N))

u = 0.5

f_t = lambda t,x: np.append(
    -D.dot(x[:-1]) + A.dot(s_xu(x[:-1],x[-1])), 
    epsilon)
res = solve_ivp(
    fun=f_t, 
    t_span=[0,10090], 
    y0=np.append(-2 + np.random.rand(N)/10, -1), 
    method="BDF")

We find a reasonable index of the solution array before the values become too large.

In [ ]:
k = np.min(np.where(res.y[0,:] > 2))

In [ ]:
plt.figure(figsize=(7,5))
plt.plot(res.y[0,0:k], label=r"$x_1(t)$")
plt.plot(res.y[N,0:k], label=r"$u(\mathbf{x},t)$")
plt.xlabel("$t$")
plt.title(r"Time series of $x_1$")
plt.legend()
mplt2tikz.save("dynamic_bif_ts.tex")

Plot the diagram \cref{fig_dyn_bif}.

In [ ]:
plt.figure(figsize=(7,5))
plt.plot(x_range, -phi_xu(x_range,0), color='black')
plt.scatter(res.y[0,0:k], res.y[N,0:k], color='red')
plt.xlabel(r"$x$")
plt.ylabel(r"$u$")
plt.title('Trajectory on the consensus space')
mplt2tikz.save("dynamic_bif_trajectory.tex")
plt.grid()